# Update gridVeg Point Intercepts in BigQuery

This notebook appends new point intercept data to BigQuery tables from a CSV file stored in GCS.

**Operation**: APPEND new rows (not replace entire table)

**Target Tables**:
- `gridVeg_point_intercept_vegetation` - vegetation intercept data (4 height layers)
- `gridVeg_point_intercept_ground` - ground cover intercept data

## Requirements
- Google Cloud credentials configured
- Configuration file: copy `config.example.yml` to `config.yml` and fill in your values
- Required packages: google-cloud-bigquery, google-cloud-storage, pandas, pyyaml


In [ ]:
# Import required libraries
import yaml
import pandas as pd
from pathlib import Path
from google.cloud import bigquery
from google.cloud import storage
from datetime import datetime

print("Libraries imported successfully")


In [ ]:
# Load configuration from YAML file
config_path = Path("../config.yml")

if not config_path.exists():
    raise FileNotFoundError(
        f"Configuration file not found: {config_path}\n"
        "Please copy config.example.yml to config.yml and fill in your values."
    )

with open(config_path, 'r') as f:
    config = yaml.safe_load(f)

# Extract configuration values for gridVeg point intercepts
GCS_CSV_URL = config['gridveg_point_intercepts']['gcs']['csv_url']
BACKUP_BUCKET = config['gridveg_point_intercepts']['gcs'].get('backup_bucket')
BACKUP_PREFIX = config['gridveg_point_intercepts']['gcs'].get('backup_prefix', 'backups/gridveg_point_intercepts')

BQ_TABLE_VEGETATION = config['gridveg_point_intercepts']['bigquery']['table_vegetation']
BQ_TABLE_GROUND = config['gridveg_point_intercepts']['bigquery']['table_ground']
BQ_PROJECT = config['gridveg_point_intercepts']['bigquery'].get('project')

# Verify required config values
if not GCS_CSV_URL or GCS_CSV_URL.startswith('gs://your-'):
    raise ValueError("Please configure gridveg_point_intercepts.gcs.csv_url in config.yml")
if not BQ_TABLE_VEGETATION or 'your-project' in BQ_TABLE_VEGETATION:
    raise ValueError("Please configure gridveg_point_intercepts.bigquery.table_vegetation in config.yml")
if not BQ_TABLE_GROUND or 'your-project' in BQ_TABLE_GROUND:
    raise ValueError("Please configure gridveg_point_intercepts.bigquery.table_ground in config.yml")

print("✓ Configuration loaded successfully")
print(f"  CSV URL: {GCS_CSV_URL[:60]}..." if len(GCS_CSV_URL) > 60 else f"  CSV URL: {GCS_CSV_URL}")
print(f"  Vegetation Table: {BQ_TABLE_VEGETATION}")
print(f"  Ground Table: {BQ_TABLE_GROUND}")
print(f"  Backup: gs://{BACKUP_BUCKET}/{BACKUP_PREFIX}" if BACKUP_BUCKET else "  Backup: Not configured")


In [ ]:
# Initialize clients
bq_client = bigquery.Client(project=BQ_PROJECT) if BQ_PROJECT else bigquery.Client()
storage_client = storage.Client(project=BQ_PROJECT) if BQ_PROJECT else storage.Client()

print(f"✓ Clients initialized")
print(f"  Project: {bq_client.project}")


In [ ]:
# Read CSV from GCS (new data)
print("Reading CSV from GCS...")
df_new = pd.read_csv(GCS_CSV_URL)

print(f"✓ CSV loaded successfully:")
print(f"  Rows: {len(df_new)}")
print(f"  Columns: {list(df_new.columns)}")
print(f"\nFirst few rows:")
df_new.head()
